In [ ]:
from brayns import Client
from circuitexplorer import CircuitExplorer
from bioexplorer import BioExplorer, Widgets, MovieMaker

In [ ]:
url = 'localhost:5000'
brayns = Client(url)
ce = CircuitExplorer(brayns)
be = BioExplorer(url)

In [ ]:
# root_folder = '/gpfs/bbp.cscs.ch/project/proj129/atlas/rat'
root_folder = '/media/favreau/Documents/medias/atlas/rat'
cache_folder = '%s/%s' % (root_folder, 'braynsCaches/v3/s1')
region_file = '%s/%s' % (root_folder, 'scans/waxholm/brain_region.json')
nrrd_folder_1 = '%s/%s' % (root_folder, 'scans/waxholm/brain_regions/brain_region')
obj_folder_1 = '%s/%s' % (root_folder, 'scans/waxholm/brain_regions/obj')
nrrd_folder_2 = '%s/%s' % (root_folder, 'scans/other')
obj_folder_2 = '%s/%s' % (root_folder, 'scans/other/obj')
# s1_config = '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/CircuitConfig_TC_WM'
s1_config = '%s/%s' % (root_folder, 'CircuitConfig_TC_WM')

create_cache = False

## Get region names

In [ ]:
import json

f = open(region_file)
obj = json.load(f)

regions = dict()
def get_children(node, level):
    regions[node['id']] = [level, node['name']]
    try:
        for child in node['children']:
            get_children(child, level + 1)
    except KeyError as e: 
        pass

get_children(obj, 0)

## Convert NRRD files into OBJ meshes

In [1]:
import os
import numpy as np
import nrrd
import glob
from skimage import measure
from scipy import ndimage


def mask_to_mesh_data(arr):
    dilated = ndimage.binary_dilation(arr, iterations = 1).astype(np.uint16)
    gaussian_blurred = ndimage.gaussian_filter(dilated - 0.5, sigma=3)

    # Make sure the final mesh has no side-of-box hole
    gaussian_blurred[:, :, 0] = -0.5
    gaussian_blurred[:, :, -1] = -0.5
    gaussian_blurred[:, 0, :] = -0.5
    gaussian_blurred[:, -1, :] = -0.5
    gaussian_blurred[0, :, :] = -0.5
    gaussian_blurred[-1, :, :] = -0.5

    vertices, triangles, normals, values = measure.marching_cubes(gaussian_blurred)
    return (vertices, triangles)

def export_obj(vertices, triangles, filepath, origin=[0,0,0], transform_3x3=[[1,0,0],[0,1,0],[0,0,1]]):
    M_xa = transform_3x3[0][0]
    M_ya = transform_3x3[0][1]
    M_za = transform_3x3[0][2]

    M_xb = transform_3x3[1][0]
    M_yb = transform_3x3[1][1]
    M_zb = transform_3x3[1][2]

    M_xc = transform_3x3[2][0]
    M_yc = transform_3x3[2][1]
    M_zc = transform_3x3[2][2]

    O_x = origin[0]
    O_y = origin[1]
    O_z = origin[2]

    obj_str = ""

    for v in vertices:
        v_x = v[0]
        v_y = v[1]
        v_z = v[2]
        v_x_world = v_x * M_xa + v_y * M_xb + v_z * M_xc + O_x
        v_y_world = v_x * M_ya + v_y * M_yb + v_z * M_yc + O_y
        v_z_world = v_x * M_za + v_y * M_zb + v_z * M_zc + O_z
        obj_str += "v "+str(v_x_world)+" "+str(v_y_world)+" "+str(v_z_world)+" \n"

    for t in triangles:
        obj_str += "f "+ str(int(t[2])+1) + " " + str(int(t[1])+1)+" " + str(int(t[0])+1)+" \n"

    f = open(filepath, 'w')
    f.write(obj_str)
    f.close()
  
def convert_region(nrrd_file, obj_folder):
    region_id = int(os.path.basename(nrrd_file).split('.')[0])
    region = regions[region_id]
    region_name = region[1].replace(' ', '_')
    mesh_file = '%s/%d_%s.obj' % (obj_folder, region_id, region_name)
    print('Processing <%s> region...' % region_name)

    # reading the nrrd file
    nrrd_data, nrrd_header = nrrd.read(nrrd_file)
    # origin = nrrd_header["space origin"]
    # transform_3x3 = nrrd_header["space directions"]
    print(nrrd_header)

    # Get vertices and triangles
    vertices, triangles = mask_to_mesh_data(nrrd_data)

    # Exporting the mesh as OBJ file
    export_obj(vertices, triangles, mesh_file)
    
def convert_file(nrrd_file, obj_folder):
    name = os.path.basename(nrrd_file).split('.')[0]
    mesh_file = '%s/%s.obj' % (obj_folder, name)
    if os.path.exists(mesh_file):
        return
    print('Processing <%s> region...' % mesh_file)

    # reading the nrrd file
    nrrd_data, nrrd_header = nrrd.read(nrrd_file)
    # origin = nrrd_header["space origin"]
    # transform_3x3 = nrrd_header["space directions"]

    # Get vertices and triangles
    vertices, triangles = mask_to_mesh_data(nrrd_data)

    # Exporting the mesh as OBJ file
    export_obj(vertices, triangles, mesh_file)

In [ ]:
'''Waxholm'''
nrrd_files = glob.glob(nrrd_folder_1 + '/*.nrrd')
for nrrd_file in nrrd_files:
    convert_region(nrrd_file, obj_folder_1)

In [ ]:
'''Other'''
nrrd_files = glob.glob(nrrd_folder_2 + '/*.nrrd')
for nrrd_file in nrrd_files:
    try:
        convert_file(nrrd_file, obj_folder_2)
    except Exception as e:
        print(e)

## Load region meshes

In [ ]:
import glob
import os

mesh_files = glob.glob(obj_folder_1 + '/*.obj')
mesh_models = list()
for mesh_file in mesh_files:
    region_id = int(os.path.basename(mesh_file).split('_')[0])
    
    region = regions[region_id]
    region_level = region[0]
    region_name = region[1]
    if region_level == 3 and region_name.find('nerve') == -1:
        model = brayns.add_model(path=mesh_file, name=region_name)
        mesh_models.append(model)

In [ ]:
for model in mesh_models:
    tf = model['transformation']
    tf['scale'] = [10, 10, 10]
    tf['translation'] = [0,-600,-450]
    brayns.update_model(id=model['id'], transformation=tf)

In [ ]:
def set_model_color(model_id, color, intensity=1.0, opacity=1.0, shading_mode=ce.SHADING_MODE_DIFFUSE):
    simulation_data_casts=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    emissions=list()
    clips=list()
    user_parameters=list()
    
    material_ids = ce.get_material_ids(model_id)['ids']
    nb_materials = len(material_ids)
    for i in range(nb_materials):
        
        if shading_mode==ce.SHADING_MODE_PERLIN:
            opacities.append(0.5)
            refraction_indices.append(1.5)
            specular_exponents.append(50)
            glossinesses.append(0.5)
            user_parameters.append(0.001)
        elif shading_mode==ce.SHADING_MODE_ELECTRON:
            opacities.append(0.75)
            refraction_indices.append(1.0)
            specular_exponents.append(5)
            glossinesses.append(1.0)
            user_parameters.append(1.0)
        elif shading_mode==ce.SHADING_MODE_DIFFUSE:
            opacities.append(opacity)
            refraction_indices.append(1.0)
            specular_exponents.append(50)
            glossinesses.append(1.0)
            user_parameters.append(1.0)
        else:
            opacities.append(1.0)
            refraction_indices.append(1.0)
            specular_exponents.append(50)
            glossinesses.append(1.0)
            user_parameters.append(1.0)

        c=color
        reflection_indices.append(0.0)
        diffuse_colors.append([c[0] * intensity,c[1] * intensity,c[2] * intensity])
        specular_colors.append([c[0] * intensity,c[1] * intensity,c[2] * intensity])
        simulation_data_casts.append(False)
        shading_modes.append(shading_mode)
        emissions.append(0)
        clips.append(True)

    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        simulation_data_casts=simulation_data_casts,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices)

In [ ]:
import seaborn as sns
nb_models = len(mesh_models)
palette = sns.color_palette('rainbow_r', nb_models)
for i in range(nb_models):
    model = mesh_models[i]
    model_id = model['id']
    ce.set_material_extra_attributes(model_id)
    set_model_color(model_id, palette[i], 
                    shading_mode=ce.SHADING_MODE_DIFFUSE,
                    opacity=0.75)

## Widgets

In [ ]:
w = Widgets(be)
w.display_model_visibility()

## S1 Circuit

### Somas only

In [ ]:
somas_models = list()
for target in ['S1DZO', 'S1DZ', 'S1FL', 'S1HL', 'S1J', 'S1Sh', 'S1Tr', 'S1ULp']:
    print('Loading target ' + target)
    cache_path = '%s/somas/%s.brayns' % (cache_folder, target)
    try:
        if create_cache:
            model = ce.load_circuit(
                name=target,
                load_soma=True, radius_multiplier=5.0,
                use_sdf=False, load_axon=False, load_apical_dendrite=False, load_dendrite=False,
                circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NONE,
                density=100, path=s1_config, targets=[target])
            ce.save_model_to_cache(
                model_id=model['id'],
                path=cache_path
            )
            somas_models.append(model)
        else:
            model = ce.load_circuit(name=target, path=cache_path)
            somas_models.append(model)
            
            if False:
                # Export meshes from point cloud of somas
                # Taubin smooth lambda=2, mu=1, iterations=100
                off_path = '%s/somas/%s.off' % (cache_folder, target)
                ce.save_model_to_mesh(model_id=model['id'], path=off_path, density=5, radius_multiplier=1000.0, shrink_factor=0.2, skin=False)
    except Exception as e:
        print(e)

In [ ]:
somas_models = list()
for target in ['S1DZO', 'S1DZ', 'S1FL', 'S1HL', 'S1J', 'S1Sh', 'S1Tr', 'S1ULp']:
    print('Loading target ' + target)
    path = '%s/somas/off/%s.off' % (cache_folder, target)
    try:
        model = brayns.add_model(name=target, path=path)
        somas_models.append(model)
    except Exception as e:
        print(e)

In [ ]:
import seaborn as sns
nb_models = len(somas_models)
palette = sns.color_palette('rainbow', nb_models)
for i in range(nb_models):
    model = somas_models[i]
    model_id = model['id']
    ce.set_material_extra_attributes(model_id)
    set_model_color(model_id, palette[i],
                    shading_mode=ce.SHADING_MODE_DIFFUSE)

In [ ]:
for model in somas_models:
    tf = model['transformation']
    tf['rotation'] = [0.0, 0.0, 1.0, 0.0]
    if model['name'] == 'S1HL':
        tf['translation'] = [-15000.0, 0.0, 0.0]
    else:
        tf['translation'] = [-7500.0, 0.0, 0.0]
    brayns.update_model(id=model['id'], transformation=tf)    

In [ ]:
for model in somas_models:
    tf = model['transformation']
    tf['rotation'] = [0.271, -0.271, 0.653, 0.653]
    tf['translation'] = [11000.0, -5000.0, -5000.0]
    brayns.update_model(id=model['id'], transformation=tf)    

### Full morphologies

In [ ]:
morphologies_models = list()
for target in ['S1DZO', 'S1DZ', 'S1FL', 'S1HL', 'S1J', 'S1Sh', 'S1Tr', 'S1ULp']:
    print('Loading target ' + target)
    try:
        cache_path = '%s/morphologies/%s.brayns' % (cache_folder, target)
        if create_cache:
            model = ce.load_circuit(
                name=target, load_soma=True, 
                use_sdf=True, load_axon=False, load_apical_dendrite=True, load_dendrite=True,
                circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NEURON_BY_LAYER,
                density=1.0, path=s1_config, targets=[target])
            ce.save_model_to_cache(
                model_id=model['id'],
                path=cache_path
            )
            morphologies_models.append(model)
        else:
            model = ce.load_circuit(name=target, path=cache_path)
            morphologies_models.append(model)        
    except Exception as e:
        print(e)

In [ ]:
import seaborn as sns
nb_models = len(morphologies_models)
palette = sns.color_palette('rainbow', nb_models)
for i in range(nb_models):
    model = morphologies_models[i]
    model_id = model['id']
    ce.set_material_extra_attributes(model_id)
    set_model_color(model_id, palette[i],
                    shading_mode=ce.SHADING_MODE_DIFFUSE, opacity=1.0)

In [ ]:
for model in morphologies_models:
    tf = model['transformation']
    tf['rotation'] = [0.707, 0.707, 0.0, 0.0]
    tf['translation'] = [-12000.0, -10000.0, 0.0]
    brayns.update_model(id=model['id'], transformation=tf)    

In [ ]:
for model in morphologies_models:
    tf = model['transformation']
    tf['rotation'] = [0.0, 0.0, 0.707, 0.707]
    tf['translation'] = [-15000.0, -10000.0, 0.0]
    brayns.update_model(id=model['id'], transformation=tf)    

In [ ]:
for model in morphologies_models:
    tf = model['transformation']
    tf['rotation'] = [0.271, -0.271, 0.653, 0.653]
    tf['translation'] = [0.0, 0.0, 0.0]
    brayns.update_model(id=model['id'], transformation=tf)    

In [ ]:
brayns.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[11158.340921621704, -93.66669235677276, 23502.815813478417],
    target=[11158.340921621704, -93.66669235677276, -4621.778547877644]    
)

In [ ]:
# height 8500
brayns.set_camera(
    orientation = [0.0, 0.0, 0.0, 1.0],
    position = [429.4863577091896, 4648.420067139924, 17051.49609983677],
    target = [429.4863577091896, 4648.420067139924, 556.4834173582194]
)

## Ground

In [ ]:
box_model = ce.add_box(
    minCorner=[-32000, -20000, -9010], maxCorner=[32000, 20000, -9000],
    color=[0.5, 0.5, 0.5, 1.0]
)

In [ ]:
model_id = box_model['id']
ce.set_material_extra_attributes(model_id)
set_model_color(model_id, [0.2, 0.2, 0.2], 
                shading_mode=ce.SHADING_MODE_NONE)

## Camera

In [ ]:
brayns.set_camera(
    orientation = [0.0, 0.0, 0.0, 1.0],
    position = [4149.881999163289, -1993.847240303193, 31419.27950858261],
    target = [4149.881999163289, -1993.847240303193, -5116.074459866043]    
)

In [ ]:
brayns.set_camera(
    orientation = [0.0, 0.0, 0.0, 1.0],
    position = [7236.659913507874, -113.2457830182348, 15587.40007899607],
    target = [7236.659913507874, -113.2457830182348, -5116.074459866043]
)

## Snaphot

In [ ]:
mm = MovieMaker(be)

In [ ]:
k = 4
mm.create_snapshot(size=[k*940, k*560], samples_per_pixel=64, base_name='rat_brain_sscx_v1', path='/home/favreau')

In [ ]:
k = 4
mm.create_snapshot(size=[k*540, k*540], samples_per_pixel=64, base_name='rat_brain_sscx_v5', path='/home/favreau')